In [19]:
import requests
import pandas as pd

from set_secrets import *
from python_roh.src.config import *

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 50)

In [20]:
ALL_EVENTS_URL = "https://www.roh.org.uk/api/events"
r = requests.get(ALL_EVENTS_URL)
data = r.json()

In [21]:
events_df = pd.DataFrame(data["data"])
included_df = pd.DataFrame(data["included"])


def pre_process_events_df(events_df, included_df):
    events_attrs = events_df.attributes.apply(pd.Series)
    events_attrs.drop(
        columns=[
            "description",
            "dateFieldOverride",
            "imageResult",
            "imageTray",
            "productionPageUrl",
            "helpInformation",
        ],
        inplace=True,
    )
    events_rels = events_df.relationships.apply(pd.Series)
    events_df = pd.concat([events_df, events_attrs, events_rels], axis=1)
    events_df.query("isCancelled != True", inplace=True)
    events_df.drop(
        columns=[
            "attributes",
            "relationships",
            "isCancelled",
            "ctaBehaviour",
            "cinemaBroadcastLink",
        ],
        inplace=True,
    )
    # remove events with no listed performances
    events_df.query("performances not in [[]]", inplace=True)
    events_df.reset_index(drop=True, inplace=True)
    events_df = events_df.explode("performances", ignore_index=True)
    events_df.locations = events_df.locations.apply(lambda x: x["data"])
    events_df = events_df.explode("locations", ignore_index=True)
    events_df.locations = events_df.locations.apply(lambda x: x["id"])
    events_df.rename(columns={"locations": "locationId"}, inplace=True)
    return events_df


def get_locations_df(included_df):
    locations_df = included_df.query("type == 'locations'").drop(
        columns=["type", "relationships"]
    )
    locations_attrs = locations_df.attributes.apply(pd.Series)
    locations_df = pd.concat([locations_df, locations_attrs], axis=1)
    locations_df.drop(columns=["attributes"], inplace=True)
    locations_df.reset_index(drop=True, inplace=True)
    locations_df.rename(columns={"id": "locationId", "title": "location"}, inplace=True)
    return locations_df


def get_performances_df(events_df):
    performances_df = events_df.performances.apply(pd.Series)
    performances_df = performances_df.assign(
        timestamp=pd.to_datetime(performances_df.date, utc=True)
    )
    # Adjust the timestamp to account for UK timezone
    performances_df.timestamp = performances_df.timestamp.dt.tz_convert("Europe/London")
    return performances_df


def enrich_events_df(events_df):
    events_df.sort_values(by=["timestamp"], inplace=True)
    events_df["date"] = events_df.timestamp.dt.date
    events_df["time"] = events_df.timestamp.dt.time
    events_df["day"] = events_df.timestamp.dt.day_name()
    events_df.drop_duplicates(
        subset=["type", "id", "location", "timestamp"], inplace=True, ignore_index=True
    )
    events_df["url"] = events_df.slug.apply(
        lambda x: f"{TICKETS_AND_EVENTS_URL}/{x}-details"
    )
    return events_df


def get_next_weeks_events(events_df, today):
    tomorrow = today + pd.Timedelta(days=1)
    next_week = today + pd.Timedelta(days=7)
    events_df = events_df.query("timestamp > @today").reset_index(drop=True)
    today_tomorrow_events_df = events_df.query("date <= @tomorrow.date()").reset_index(
        drop=True
    )
    next_week_events_df = events_df.query("date <= @next_week.date()").reset_index(
        drop=True
    )
    return today_tomorrow_events_df, next_week_events_df




In [24]:
events_df = pre_process_events_df(events_df, included_df)

locations_df = get_locations_df(included_df)
events_df = events_df.merge(locations_df, on="locationId", how="left")

performances_df = get_performances_df(events_df)
events_df = pd.concat([events_df, performances_df], axis=1)
events_df.drop(columns=["performances", "date"], inplace=True)
events_df = enrich_events_df(events_df)

today = pd.Timestamp.today(tz="Europe/London") - pd.Timedelta(hours=1)
today_tomorrow_events_df, next_week_events_df = get_next_weeks_events(events_df, today)

In [25]:
import plotly.express as px


def plot_events(events_df, today):
    """
    Plot the timeline of the upcoming events on the Main Stage
    """
    events_df_sub = events_df.query(
        "location == 'Main Stage' & date >= @today.date()"
    ).reset_index(drop=True)
    events_df_sub["Size"] = 1  # set the size of the dots
    fig = px.scatter(
        events_df_sub,
        custom_data=["title", "url"],
        x="date",
        y="time",
        color="title",
        size="Size",
        size_max=7,
        title="Royal Opera House Events",
        template="simple_white",
        hover_name="url",
    )
    # Keep first 5 characters of the y axis marks
    fig.update_yaxes(
        categoryorder="category ascending",
        showgrid=True,
        tickvals=events_df_sub.time.unique(),
        ticktext=[str(x)[:5] for x in events_df_sub.time.unique()],
        title="",
    )
    fig.update_xaxes(
        title="",
        showgrid=True,
        gridwidth=1,
        gridcolor="LightGray",
    )
    fig.update_layout(
        width=1700,
        height=500,
        margin=dict(l=0, r=0, b=0, t=40, pad=0),
    )
    fig.update_traces(
        marker=dict(line=dict(width=1, color="DarkSlateGrey")),
        selector=dict(mode="markers"),
    )
    fig.update_layout(
        hoverlabel=dict(
            font_size=16,
            font_family="Gotham",
            font_color="White",
            bgcolor="#C7102E",
        ),
        legend=dict(
            title="Title",
            title_font=dict(size=15, family="Gotham"),
            font=dict(size=13, family="Gotham"),
        ),
        title=None,
    )
    fig.layout.font.family = "Gotham"
    # Remove Size from the hover data
    fig.update_traces(
        hovertemplate="<br>".join(
            [
                "%{customdata[0]}",
                "%{x}",
                "%{y}",
            ],
        )
        + "<extra></extra>",
    )
    fig.show()

plot_events(events_df, today)

In [15]:
fig.write_image("output/ROH_events.png", scale=3)